In [128]:
from dataclasses import dataclass
import sympy
import numpy as np
import pickle
import copy
from collections import deque
from sympy.utilities import lambdify
from tqdm import tqdm
import warnings




warnings.filterwarnings("error")
rng = np.random.default_rng()

# EXTRA_FUNC_DICT = {
#     'exp': make_function(np.exp, 'exp', 1),
#     'pow': make_function(np.power, 'pow', 2)
# }

STR2SYMPY = {
    'neg': lambda x: -x,
    'abs': sympy.Abs,
    'sqrt': sympy.sqrt,
    'exp': sympy.exp,
    'log': sympy.log,
    'sin': sympy.sin,
    'cos': sympy.cos,
    'tan': sympy.tan,
    'add': lambda x, y : x + y,
    'sub': lambda x, y : x - y,
    'mul': lambda x, y : x * y,
    'div': lambda x, y : x / y,
    'pow': lambda x, y : x ** y
}

binary_functions=np.array(['add','sub','mul','div','pow'])
unary_functions=np.array(['neg','sqrt','exp','log','sin','cos','tan'])
extra_constants=np.array(['2','3','4','5','pi'])
true_variables=(['x0','x1','x2','x3','x4','x5'])

@dataclass
class ExpNode:
    kind: str
    num_children: int
    children: list
    parent: 'ExpNode' = None

def find_depth(node: ExpNode,currdepth):
    if isinstance(node, np.ndarray):
        return currdepth
    elif node.num_children==1:
        return find_depth(node.children[0],currdepth+1)
    else:
        return max(find_depth(node.children[0],currdepth+1),find_depth(node.children[1],currdepth+1))

def find_variables(root: ExpNode):
    if isinstance(root, np.ndarray):
        return root
    dir=rng.uniform(0,1)
    if dir<1/2 or root.num_children==1:
        return find_variables(root.children[0])
    elif dir>1/2 and root.num_children>1:
        return find_variables(root.children[1])


def visit(formula: ExpNode):
    tovisit=deque()
    tovisit.append((formula,None,0))
    count=0
    while tovisit:
        node,parent,j=tovisit.popleft()
        if not isinstance(node, np.ndarray):
            for i,child in enumerate(node.children):
                tovisit.append((child,node,i))
        count+=1
        if rng.integers(1,count+1)==count:
            item=node
            itemparent=parent
            itemid=j
    return item,itemparent,itemid

def cut_and_shorten(formula: ExpNode):
    cutpoint,cutparent,cutid=visit(formula)
    if cutparent is not None:
        cutparent.children[cutid]=find_variables(cutpoint)
    return formula

def cut_and_regenerate(formula: ExpNode, variables,dummy,simplified):
    cutpoint,cutparent,cutid=visit(formula)
    if cutparent is not None:
        cutparent.children[cutid]=generate_Expression(variables,dummy,simplified)
        cutparent.children[cutid].parent=cutparent
    else:
        formula=generate_Expression(variables,dummy,simplified)
    return formula

def crossover(formula1: ExpNode,formula2: ExpNode):
    cxpoint1,parent1,id1=visit(formula1)
    cxpoint2,parent2,id2=visit(formula2)
    if (parent1 is None) and (not isinstance(cxpoint2,np.ndarray)):
        return cxpoint2
    elif (parent1 is not None) and (not isinstance(cxpoint2,np.ndarray)):
        cxpoint2.parent=parent1
        parent1.children[id1]=cxpoint2
        return formula1
    elif parent1 is not None:
        parent1.children[id1]=cxpoint2
        return formula1
    else:
        return formula1



def choose_function():
    fn_choice=rng.uniform(0,1)
    children=0
    if fn_choice>7/12:
        kind=rng.choice(binary_functions,1)
        children=2
    else:
        kind=rng.choice(unary_functions,1)
        children=1
    return kind[0],children

def generate_subtree(root,depth,max_depth,variables):
    if depth==max_depth:
        for n in range(root.num_children):
            var=rng.choice(variables,1)
            # if var[0]=='const':
            root.children.append(var)
    else:
        for n in range(root.num_children):
            fn,children=choose_function()
            child=ExpNode(fn,children,[],root)
            generate_subtree(child,depth+1,max_depth,variables)
            root.children.append(child)


def generate_Expression(variables: np.ndarray,dummy=True,simplified=False):
    depth=rng.integers(1,3)
    num_variables=rng.integers(1,variables.shape[0]+1)
    #check if replacement mayu be usefull (the same variable is used twice in different fragments of the expression)
    # we may choose just one variable then
    if dummy:
        used_variables=rng.choice(variables,num_variables,replace=False)
    else:
        used_variables=variables
    #print(num_variables,used_variables)
    #using uniform generation
    d=0
    curr_fn,kids=choose_function()
    root=ExpNode(curr_fn,kids,[])
    if simplified:
        generate_subtree(root,1,depth,used_variables)
    else:
        generate_subtree(root,1,depth,np.concatenate([used_variables,extra_constants]))
    return root

        
# def parse_Expression(expr: ExpNode):
#     if type(expr)==np.ndarray:
#         return sympy.symbols(expr[0])
#     else:
#         print(expr.kind)
#         print((parse_Expression(expr.children[i]) for i in range(expr.num_children)))
#         return STR2SYMPY[expr.kind](parse_Expression(expr.children[i]) for i in range(expr.num_children))

def parse_Expression(expr: ExpNode):
    if isinstance(expr, np.ndarray):
        return sympy.symbols(expr[0])
    elif expr.kind in STR2SYMPY:
        # Parse children recursively
        parsed_children = [parse_Expression(child) for child in expr.children]
        if expr.kind == 'div' and len(parsed_children) == 2:
            # Check for division by zero
            denominator = parsed_children[1]
            if denominator == 0:
                raise ZeroDivisionError(f"Division by zero detected in expression: {expr}")
        # Apply the corresponding sympy function/operator
        return STR2SYMPY[expr.kind](*parsed_children)
    else:
        raise ValueError(f"Unsupported expression kind: {expr.kind}")
    

    # except RuntimeWarning:
        #     return np.inf
    # print(type(fn))
    # print(data.shape)
    # print(data[:,nums].shape)
    # print(fn(data[:,nums]))
    # print(data[:,[data.shape[1]-1]])
    #print(np.sqrt((fn(data[:,nums])-data[:,[data.shape[1]-1]])*(fn(data[:,nums])-data[:,[data.shape[1]-1]])))
    # temp=[]
    # indices=[temp.extend(num) for num in nums]
    #print(nums)
    


In [129]:
class SGA:
    def __init__(self,pop_size,children_size,crossover_prob,mutation_prob,num_iter,variables,data,dummy=False,simplified=False):
        self.pop_size=pop_size
        self.crossover_prob=crossover_prob
        self.mutation_prob=mutation_prob
        self.num_iter=num_iter
        self.variables=variables
        #self.objective_function=objective_function
        self.dummy=dummy
        self.data=data
        self.children_size=children_size
        self.simplified=simplified

    def initial_population(self):
        self.parent_population=[generate_Expression(self.variables,self.dummy,self.simplified) for i in range(self.pop_size)]
        self.best_individual=self.parent_population[0]
        #a litt;e shenanigans
        self.best_objective_value = min([self.objective_function(pop,self.data) for pop in self.parent_population])
    
    def objective_function(self,expression,data):

        try:
            parsed_expression=parse_Expression(expression)
        except:
            return np.inf
        
        variables=[_ for _ in parsed_expression.atoms() if type(_) is sympy.Symbol and str(_) in true_variables]
        nums=[int(str(_)[1]) for _ in variables]
        #print(parsed_expression,nums)
        try:
            fn=lambdify(variables,parsed_expression)
            if len(nums)==0:
                return np.inf
            
            score=np.sum(np.sqrt((fn(data[:,nums])-data[:,[data.shape[1]-1]])*(fn(data[:,nums])-data[:,[data.shape[1]-1]])))
            depth=find_depth(expression,0)
            if depth>3*data.shape[1]:
                score=score*depth
            if len(nums)<data.shape[1] and self.dummy==False:
                return (data.shape[1]-len(nums))*score
            else:
                return score
        except:
            return np.inf

    def fit(self):
        for t in tqdm(range(self.num_iter)):
            self.children_population=[]
            
            for i in range(self.pop_size-1):
                if rng.uniform(0,1)<self.crossover_prob:
                    #check here if copy is deep enough
                    self.children_population.append(crossover(copy.deepcopy(self.parent_population[i]),copy.deepcopy(self.parent_population[i+1])))
                else:
                    self.children_population.append(self.parent_population[i])
            #print(len(self.children_population))

            for i in range(self.children_size-1):
                if rng.uniform(0,1)<self.mutation_prob:
                    self.children_population[i]=cut_and_regenerate(self.children_population[i],self.variables,self.dummy,self.simplified)
                elif rng.uniform(0,1)<self.mutation_prob:
                    self.children_population[i]=cut_and_shorten(self.children_population[i])
                    
            total_pop=self.parent_population+self.children_population
            self.fitness_values=[self.objective_function(pop,self.data) for pop in total_pop]
            self.parent_population=[x for _, x in sorted(zip(self.fitness_values, total_pop), key=lambda pair: pair[0])][:self.pop_size]

            if min(self.fitness_values)<self.best_objective_value:
                self.best_objective_value=min(self.fitness_values)
                self.best_individual=self.parent_population[0]
            
            print(parse_Expression(self.best_individual),self.best_objective_value)
            #print([parse_Expression(pop,)])
            sample=[]
            for i in range(10):
                try:
                    sample.append(parse_Expression(self.parent_population[i]))
                except:
                    print('div by 0')
            print(sample)

        return parse_Expression(self.best_individual)


In [25]:
t=np.array(['x1','x2','x3'],dtype=str)
res=generate_Expression(t)
print(res)

1 ['x2']
ExpNode(kind=np.str_('pow'), num_children=2, children=[ExpNode(kind=np.str_('add'), num_children=2, children=[array(['4'], dtype='<U2'), array(['4'], dtype='<U2')], parent=...), ExpNode(kind=np.str_('mul'), num_children=2, children=[array(['4'], dtype='<U2'), array(['x2'], dtype='<U2')], parent=...)], parent=None)


In [6]:
print(visit(res))


(ExpNode(kind=np.str_('log'), num_children=1, children=[array(['4'], dtype='<U2')], parent=ExpNode(kind=np.str_('pow'), num_children=2, children=[ExpNode(kind=np.str_('exp'), num_children=1, children=[array(['x'], dtype='<U2')], parent=...), ...], parent=None)), ExpNode(kind=np.str_('pow'), num_children=2, children=[ExpNode(kind=np.str_('exp'), num_children=1, children=[array(['x'], dtype='<U2')], parent=...), ExpNode(kind=np.str_('log'), num_children=1, children=[array(['4'], dtype='<U2')], parent=...)], parent=None), 1)


In [55]:
symbolic=parse_Expression(res)
print(type(parse_Expression(res)))
variables=[_ for _ in symbolic.atoms() if type(_) is sympy.Symbol and str(_) in true_variables]
if len(variables)>0:
    print(type(variables[0]),variables[0])

<class 'sympy.core.power.Pow'>
<class 'sympy.core.symbol.Symbol'> x2


In [86]:
# with open('srsd/medium_set/true_eq/feynman-i.8.14.pkl','rb') as eq:
#     equaion=pickle.load(eq)
#     print(equaion,type(equaion))

# arr=np.loadtxt('srsd/medium_set/test/feynman-i.8.14.txt')
# print(arr)
# arr1=np.loadtxt('srsd/medium_set/test/feynman-i.34.8.txt')
# print(arr1)

#dummy variable amounts disagree with dataset decription
data=np.loadtxt('srsd/easy_set_dummy/train/feynman-i.12.5.txt')
print(data.shape)
with open('srsd/easy_set_dummy/true_eq/feynman-i.12.5.pkl','rb') as eq:
    equaion=pickle.load(eq)
    print(equaion,type(equaion))



(8000, 5)
x2*x3 <class 'sympy.core.mul.Mul'>


In [113]:
testing=SGA(100,100,0.95,0.3,100,np.array(['x0','x1','x2','x3']),data,dummy=True)
testing.initial_population()
testing.fit()

  1%|          | 1/100 [00:40<1:06:47, 40.48s/it]

exp(x3) 35341.04400568461


  2%|▏         | 2/100 [00:45<31:52, 19.51s/it]  

exp(x3) 35341.04400568461


  3%|▎         | 3/100 [00:54<23:36, 14.61s/it]

...

 99%|█████████▉| 99/100 [24:29<00:29, 29.34s/it]

-2*(x0*tan(4*5))**(1/4) 35340.29096439666


100%|██████████| 100/100 [24:29<00:00, 14.70s/it]

-2*(x0*tan(4*5))**(1/4) 35340.29096439666


ExpNode(kind=np.str_('neg'), num_children=1, children=[ExpNode(kind=np.str_('mul'), num_children=2, children=[array(['2'], dtype='<U2'), ExpNode(kind=np.str_('sqrt'), num_children=1, children=[ExpNode(kind=np.str_('sqrt'), num_children=1, children=[ExpNode(kind=np.str_('mul'), num_children=2, children=[ExpNode(kind=np.str_('tan'), num_children=1, children=[ExpNode(kind=np.str_('mul'), num_children=2, children=[array(['5'], dtype='<U2'), array(['4'], dtype='<U2')], parent=...)], parent=...), array(['x0'], dtype='<U2')], parent=...)], parent=...)], parent=...)], parent=...)], parent=None)

In [116]:
with open('srsd/medium_set/true_eq/feynman-i.8.14.pkl','rb') as eq:
    equaion=pickle.load(eq)
    print(equaion,type(equaion))

arr=np.loadtxt('srsd/medium_set/test/feynman-i.8.14.txt')
print(arr)
testing=SGA(100,100,0.95,0.3,100,np.array(['x0','x1','x2','x3']),arr,dummy=False)
testing.initial_population()
testing.fit()


# print(rng.integers(1,2))
# w=deque()
# w.append('a')
# w.append(17)
# print(w)

sqrt((x0 - x1)**2 + (x2 - x3)**2) <class 'sympy.core.power.Pow'>
[[ 2.00293401  0.76615789 -1.16301986 -2.29924894  1.67947364]
 [-0.1516021  -0.315528    4.28046425 -0.16175568  4.44524347]
 [ 7.22516759 -0.60686739  5.98877255 -2.48898992 11.54180351]
 ...
 [-0.38752721 -3.80981155 -4.00705763 -9.805081    6.73268929]
 [ 0.15153997  1.39060106 -0.13542526 -5.74073294  5.74062249]
 [-3.96895032 -3.69390316  2.53040955 -1.05597386  3.59691491]]


  1%|          | 1/100 [00:17<28:42, 17.40s/it]

exp(pi) 7362.35645742886


  2%|▏         | 2/100 [00:20<14:23,  8.81s/it]

5 - x2 7323.74971794977


...

 99%|█████████▉| 99/100 [02:35<00:01,  1.74s/it]

5 - sin(sin(sin(5 - sin(exp(3) + exp(x0/3))) + cos(x0/3)) + sin(sin(cos(x0/3)))) - sin(sin(sin(sin(cos(3)))) + sin(sin(cos(x0/3)))) - sin(sin(4) + sin(sin(-2 + 5 + sin(sin(cos(x0/3))))) + sin(sin(cos(x0/3)))) - sin(cos(x0/3)) 5200.194624801401


100%|██████████| 100/100 [02:36<00:00,  1.57s/it]

5 - sin(sin(sin(5 - sin(exp(3) + exp(x0/3))) + cos(x0/3)) + sin(sin(cos(x0/3)))) - sin(sin(sin(sin(cos(3)))) + sin(sin(cos(x0/3)))) - sin(sin(4) + sin(sin(-2 + 5 + sin(sin(cos(x0/3))))) + sin(sin(cos(x0/3)))) - sin(cos(x0/3)) 5200.194624801401


5 - sin(sin(sin(5 - sin(exp(3) + exp(x0/3))) + cos(x0/3)) + sin(sin(cos(x0/3)))) - sin(sin(sin(sin(cos(3)))) + sin(sin(cos(x0/3)))) - sin(sin(4) + sin(sin(-2 + 5 + sin(sin(cos(x0/3))))) + sin(sin(cos(x0/3)))) - sin(cos(x0/3))

In [ ]:
#print(type(t[0]))
#sympy.parse_expr('log(x1)/sin(pi)')
#before adding depth regularization
testing=SGA(100,100,0.95,0.3,100,np.array(['x0','x1','x2','x3']),arr,dummy=False)
testing.initial_population()
testing.fit()

  1%|          | 1/100 [00:23<38:55, 23.60s/it]

-sin(x1) 7271.026189509979


  2%|▏         | 2/100 [00:37<29:02, 17.78s/it]

-sin(x1) 7271.026189509979


  3%|▎         | 3/100 [00:40<18:09, 11.23s/it]

...

 99%|█████████▉| 99/100 [04:26<00:08,  8.63s/it]

2 + 3 + sin(2 + 3**(3**(sin(2 + 2**(2*2 + 3**(3**(sin(2 + sin(sqrt(3))**(sqrt(3))/x0)/2 + sin(2 + sin(sqrt(3))**(2 + sin(cos(5)) + sin(sqrt(3))**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/x0)/2)/2 + sin(2 + sin(sqrt(3))**(3**(sin(sqrt(3))/2 + sin(2 + sin(sqrt(3))**(2 + log(sin(3))/sqrt(exp(sqrt(3))))/x0)/2)/x0)/x0)/2)/x0 + x0)/x0)/2 + sin(2 + sin(5**x3)**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/2)/2 + sin(2 + sin(sqrt(3))**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/2)/x0) + sin(2 + 3**(sin(2 + sin(sqrt(3))**(2*2 + sin(2 + 3**(3**(sin(2 + sin(sqrt(3))**(2*2 + x0)/x0)/2 + sin(2 + sin(sqrt(3))**(2 + 3 + sin(cos(5)))/x0)/2)/2 + sin(2 + sin(sqrt(3))**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/2)/x0))/x0)/2 + sin(cos(2 + 2/x0))/2)/x0) 5386.910248857201


100%|██████████| 100/100 [04:35<00:00,  2.76s/it]

2 + 3 + sin(2 + 3**(3**(sin(2 + 2**(2*2 + 3**(3**(sin(2 + sin(sqrt(3))**(sqrt(3))/x0)/2 + sin(2 + sin(sqrt(3))**(2 + sin(cos(5)) + sin(sqrt(3))**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/x0)/2)/2 + sin(2 + sin(sqrt(3))**(3**(sin(sqrt(3))/2 + sin(2 + sin(sqrt(3))**(2 + log(sin(3))/sqrt(exp(sqrt(3))))/x0)/2)/x0)/x0)/2)/x0 + x0)/x0)/2 + sin(2 + sin(5**x3)**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/2)/2 + sin(2 + sin(sqrt(3))**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/2)/x0) + sin(2 + 3**(sin(2 + sin(sqrt(3))**(2*2 + sin(2 + 3**(3**(sin(2 + sin(sqrt(3))**(2*2 + x0)/x0)/2 + sin(2 + sin(sqrt(3))**(2 + 3 + sin(cos(5)))/x0)/2)/2 + sin(2 + sin(sqrt(3))**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/2)/x0))/x0)/2 + sin(cos(2 + 2/x0))/2)/x0) 5386.910248857201


2 + 3 + sin(2 + 3**(3**(sin(2 + 2**(2*2 + 3**(3**(sin(2 + sin(sqrt(3))**(sqrt(3))/x0)/2 + sin(2 + sin(sqrt(3))**(2 + sin(cos(5)) + sin(sqrt(3))**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/x0)/2)/2 + sin(2 + sin(sqrt(3))**(3**(sin(sqrt(3))/2 + sin(2 + sin(sqrt(3))**(2 + log(sin(3))/sqrt(exp(sqrt(3))))/x0)/2)/x0)/x0)/2)/x0 + x0)/x0)/2 + sin(2 + sin(5**x3)**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/2)/2 + sin(2 + sin(sqrt(3))**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/2)/x0) + sin(2 + 3**(sin(2 + sin(sqrt(3))**(2*2 + sin(2 + 3**(3**(sin(2 + sin(sqrt(3))**(2*2 + x0)/x0)/2 + sin(2 + sin(sqrt(3))**(2 + 3 + sin(cos(5)))/x0)/2)/2 + sin(2 + sin(sqrt(3))**(2 + 3 + sin(cos(sin(2 + 2/x0))))/x0)/2)/x0))/x0)/2 + sin(cos(2 + 2/x0))/2)/x0)

In [123]:
testing=SGA(100,100,0.95,0.3,100,np.array(['x0','x1','x2','x3']),arr,dummy=False)
testing.initial_population()
testing.fit()

  1%|          | 1/100 [00:14<23:52, 14.47s/it]

2 + 3 14805.311954711597


  2%|▏         | 2/100 [00:27<22:42, 13.91s/it]

5 + x3 14560.162709228785


  3%|▎         | 3/100 [00:31<14:40,  9.08s/it]

...

 99%|█████████▉| 99/100 [02:28<00:01,  1.45s/it]

5 + cos(4 + 5 + cos(exp(-2 + 4)) - x2/3) + cos(exp(2)) 10806.040167396182


100%|██████████| 100/100 [02:28<00:00,  1.49s/it]

5 + cos(3 + 4) + cos(4 + 5 + cos(exp(2)) - x2/3) 10783.976434999158


5 + cos(3 + 4) + cos(4 + 5 + cos(exp(2)) - x2/3)

In [125]:
testing=SGA(100,100,0.95,0.3,100,np.array(['x0','x1','x2','x3']),arr,dummy=False)
testing.initial_population()
testing.fit()

  1%|          | 1/100 [00:17<29:15, 17.73s/it]

3 + x1 15451.468827629196
[3 + x1, 3 - x1, 2 + x0, (3 + x1)/sqrt(5), 2 - x3, cos(cos(x0)), cos(cos(x1)), cos(cos(x1)), cos(x0), x3]


  2%|▏         | 2/100 [00:30<24:30, 15.01s/it]

3 + x1 15451.468827629196
[3 + x1, 3 - x1, 3 - x3, 2 + x0, (3 + x1)/sqrt(5), (3 + x1)/sqrt(5), 2 - x3, (2 + x1)/sqrt(5), cos(cos(cos(x0))), cos(cos(cos(cos(x1))))]


  3%|▎         | 3/100 [00:32<14:34,  9.02s/it]

...

 99%|█████████▉| 99/100 [02:33<00:01,  1.78s/it]

5 + cos(3 + (-x0 + cos((5 + (-x0 + cos(5))/sqrt(5))/5))/sqrt(5)) + cos(3 + (-x0 + cos(5))/sqrt(5 + cos(cos(5)))) + cos(cos(4/sqrt(5))) 10490.981798177903
[5 + cos(3 + (-x0 + cos((5 + (-x0 + cos(5))/sqrt(5))/5))/sqrt(5)) + cos(3 + (-x0 + cos(5))/sqrt(5 + cos(cos(5)))) + cos(cos(4/sqrt(5))), 5 + cos(5) + cos(3 + (-x0 + cos(5))/sqrt(5)) + cos(3 + (cos(x0) + (-x0 + cos(5))/5**(1/4))/sqrt(5)) + cos(log(2)), 5 + cos(3 + (-x0 + cos(5))/sqrt(5)) + cos(3 + (-x0 + cos((5 + (-x0 + cos(5))/sqrt(5))/5))/sqrt(5)) + cos(cos(4/sqrt(5))), 5 + cos(5 + cos(3 + (3 + x3)/3)) + cos(5 + cos(3 + 2) + cos(3 + x3/sqrt(5 + cos(3 + x3)))), 5 + cos(3 + (-x0 + cos(5))/sqrt(5)) + cos(3 + (cos(5) + (-x0 + cos(5))/sqrt(3 + (cos(5) + (-x0 + cos(5))/sqrt(5))/sqrt(5)))/sqrt(5)) + cos(5 + cos(5)) + cos(log(2)), 5 + cos(5) + cos(3 + (-x0 + cos(5))/sqrt(5)) + cos(3 + (cos(5) + (-x0 + cos(5))/sqrt(3))/sqrt(5)) + cos(log(2)), 5 + cos(5) + cos(3 + (-x0 + cos(5))/sqrt(5)) + cos(3 + (cos(cos(x0)) + (-x0 + cos(5))/sqrt(3 + (cos(5

100%|██████████| 100/100 [02:34<00:00,  1.55s/it]

5 + cos(5) + cos(3 + (-x0 + cos((5 + (-x0 + cos(5))/sqrt(5))/5))/sqrt(5)) + cos(3 + (cos(x0) + (-x0 + cos(5))/5**(1/4))/sqrt(5)) + cos(log(2)) 10486.466698475215
[5 + cos(5) + cos(3 + (-x0 + cos((5 + (-x0 + cos(5))/sqrt(5))/5))/sqrt(5)) + cos(3 + (cos(x0) + (-x0 + cos(5))/5**(1/4))/sqrt(5)) + cos(log(2)), 5 + cos(3 + (-x0 + cos((5 + (-x0 + cos(5))/sqrt(5))/5))/sqrt(5)) + cos(3 + (-x0 + cos(5))/sqrt(5 + cos(cos(5)))) + cos(cos(4/sqrt(5))), 5 + cos(5) + cos(3 + (-x0 + cos(5))/sqrt(5)) + cos(3 + (cos(5) + (-x0 + cos((5 + (-x0 + cos(5))/sqrt(5))/sqrt(5)))/sqrt(3 + (cos(2*x0) + (-5 + cos(5))/sqrt(5))/sqrt(5)))/sqrt(5)) + cos(log(2)), 5 + cos(5) + cos(3 + (-x0 + cos(5))/sqrt(5)) + cos(3 + (cos(x0) + (-x0 + cos(5))/5**(1/4))/sqrt(5)) + cos(log(2)), 5 + cos(3 + (-x0 + cos(5))/sqrt(5)) + cos(3 + (-x0 + cos((5 + (-x0 + cos(5))/sqrt(5))/5))/sqrt(5)) + cos(cos(4/sqrt(5))), 5 + cos(5 + cos(3 + (3 + x3)/3)) + cos(5 + cos(3 + 2) + cos(3 + x3/sqrt(5 + cos(3 + x3)))), 5 + cos(3 + (-x0 + cos(5))/sqrt(5)

5 + cos(5) + cos(3 + (-x0 + cos((5 + (-x0 + cos(5))/sqrt(5))/5))/sqrt(5)) + cos(3 + (cos(x0) + (-x0 + cos(5))/5**(1/4))/sqrt(5)) + cos(log(2))

In [130]:
testing=SGA(100,100,0.95,0.3,100,np.array(['x0','x1','x2','x3']),arr,dummy=False,simplified=True)
testing.initial_population()
testing.fit()

  2%|▏         | 2/100 [00:00<00:10,  9.07it/s]

x1**2 18351.009661017008
[exp(sin(x3)), exp(sin(x3)), cos(cos(x1)), cos(cos(x1)), 1 - cos(x2), sin(cos(cos(x3))), sin(sqrt(exp(x0))), cos(exp(-x1)), sin(exp(x0)), cos(x0)]
exp(cos(x1)) 17140.733225310243
[exp(cos(x1)), exp(sin(x3)), exp(sin(x3)), cos(cos(cos(cos(x1)))), cos(cos(x3)), cos(cos(x1)), cos(cos(x1)), 1 - cos(x2), sin(cos(cos(x3))), sin(cos(cos(x0)))]
exp(cos(x1)) 17140.733225310243
[exp(cos(x1)), exp(cos(x1)), exp(sin(x3)), exp(sin(x3)), exp(sin(x3)), cos(exp(-x1**2)), cos(cos(cos(x3))), cos(cos(cos(cos(x1)))), cos(cos(cos(cos(x1)))), cos(cos(x3))]


  6%|▌         | 6/100 [00:00<00:09,  9.89it/s]

exp(cos(x1)) 17140.733225310243
[exp(cos(x1)), exp(cos(x1)), exp(sin(x3)), exp(sin(x3)), exp(sin(x3)), cos(exp(-x1**2)), cos(cos(cos(x3))), cos(cos(cos(x3))), cos(cos(cos(cos(x1)))), cos(cos(cos(cos(x1))))]
exp(sin(exp(sin(x3)))) 15699.47099184717
[exp(sin(exp(sin(x3)))), exp(cos(x1)), exp(cos(x1)), exp(sin(x3)), exp(sin(x3)), exp(sin(x3)), cos(exp(-x1**2)), cos(exp(-x0**2)), cos(sin(tan(x1))), cos(cos(cos(sin(x2))))]
exp(cos(exp(-x1**2))) 14564.788887425857
[exp(cos(exp(-x1**2))), exp(sin(exp(sin(x1)))), exp(sin(exp(sin(x3)))), exp(exp(sin(x3))), exp(sin(exp(x0))), exp(cos(x1)), exp(cos(x1)), exp(sin(x3)), exp(sin(x3)), exp(sin(x3))]


  8%|▊         | 8/100 [00:00<00:10,  8.81it/s]

...

 99%|█████████▉| 99/100 [00:25<00:00,  3.21it/s]

exp(x0 - x3) 11173.942582830648
[exp(exp(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))), exp(exp(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))), exp(exp(sin(sin(sin(sin(exp(-sin(sin(exp(x3))))))))))]


100%|██████████| 100/100 [00:25<00:00,  3.85it/s]

exp(x0 - x3) 11173.942582830648
[exp(exp(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3))))))))))), exp(exp(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))))), exp(exp(sin(sin(sin(sin(exp(-sin(sin(exp(x3)))))))))), exp(exp(sin(sin(sin(sin(exp(-sin(sin(exp(x3))))))))))]


exp(x0 - x3)